# datathon (IBM) 19-Oct-2024
## treetastic

Helps to decide where to plant trees in cities and parks based on one
or more images.

### purpose

- Noise reduction
- Air quality
- Aesthetics
- Mental health
- Community space

### workflow

1. Image (geotagged) and budget provided
    - Images should look down the middle of street
2. Images passed through API to Python function
3. Images analysed and determines list of ideal pixels for new trees
    - Assigned with score for improvement to area
4. Program iteratively decides which trees are best for an area
5. List of trees and images are returned to the interface

### tech stack

- Python (interacting with AI)
    - OpenCV for image analysis

### Task 1: Image analysis
Using the OpenCV library

In [29]:
from os import listdir
from os.path import isfile, join
import cv2
import matplotlib.pyplot as plt

# find the files
local_path = "/Users/jayambadkar/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Personal/Datathon/Photos"
getFullPhotoNames = [join(local_path, f) for f in listdir(local_path) if isfile(join(local_path, f))]


In [30]:

# read images as greyscale with library and store in a list
# images = [[cv2.imread(join(local_path, im_path), cv2.IMREAD_GRAYSCALE), im_path] for im_path in getFullPhotoNames]


### Post-processing
The ML model now outputs the space and buildings based on an input image.

So we are free to continue.

In [58]:
from trees import Tree, getTreeTypes
species = trees.getTreeTypes()


In [84]:
from nearbyPollution import nearbyPollution


def giveTreeLocationInPic(imagetextFilePath, imageFilePath):
    if (nearbyPollution < 3):
        return []
    lines = []
    with open(imagetextFile, 'r') as file:
        lines = file.readlines()
    newlines = []
    for line in lines:
        newlines.append(line.strip("\n"))
    newlines = [[float(x) for x in l.split(" ")] for l in newlines]
    
    # ok so each identified region is ready to be processed
    # [0] is type (0=building, 1=tree, 2=space),
    # [1] = centre x, [2] = centre y, [3]=width, [4]=height

    existingTrees, buildingsAround = 0
    possTrees= [] # list of lists, with each tree pos given by [x, y, type]
    cost = 0
    
    for region in newlines:
        if (region[0] == 0):
            buildingsAround += 1
        elif (region[0] == 1):
            existingTrees += 1
        else:
            for t in range(len(species)-1, -1):
                if (species[t].width < region.width and species[t].height < region.height):
                    possTrees.append([region[1], region[2], species[t].name])
                    cost += species[t].cost
                    break
            
    if (existingTrees > len(possTrees)):
        return []

    print(f"Net benefit for {imageFilePath} is {buildingsAround * len(possTrees)} at a cost of approximately £{cost}.\nReturned the positions and types of the relevant recommended trees.")
    return [possTrees, cost, buildingsAround * len(possTrees)]
